Author: Jiaying (Claire) Yang

This implementation is based on pyQuil version 0.2.4. It is exactly the same experiment with the QISKit implementation, thus there is no detailed explanation here.

The input in this implementation is  $A=\begin{pmatrix}
2 & 1\\ 
1 & 2
\end{pmatrix}$ and $\vec{b}=\begin{pmatrix}
0\\ 
1
\end{pmatrix}$

In [1]:
from pyquil import Program, get_qc
from pyquil.gates import *
from math import pi
import numpy as np

I find that there is a quantum gate RY in pyQuil, but it is only for 1 qubit. What I need is the controlled-RY, which can input two qubits as the control qubit and target qubit. Thus we can use the below cell to define the quantum gate by providing the unitary matrix (reference: http://docs.rigetti.com/en/stable/basics.html#defining-new-gates).

Another way to solve this problem is by using the `CONTROLLED` keyword, which the SDK tools will happily consume, but pyQuil is still learning to recognize: https://github.com/rigetti/pyquil/pull/808.

In [2]:
# First we define the controlled RY gate from a matrix
from pyquil.parameters import Parameter, quil_exp, quil_sin, quil_cos
from pyquil.quilbase import DefGate

# Define the new gate controlled-RY from a matrix
theta = Parameter('theta')
cry = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, quil_cos(theta / 2), -1 * quil_sin(theta / 2)],
    [0, 0, quil_sin(theta / 2), quil_cos(theta / 2)]
])

gate_definition = DefGate('CRY', cry, [theta])
CRY = gate_definition.get_constructor()

The quantum circuit is:

In [3]:
p = Program() 
p += gate_definition

p += X(3) # Initilize input b

### Phase estimation ###
## Superposition
p += H(1)
p += H(2)
## Controlled-U
p += PHASE(pi/2, 2)
p += PHASE(pi, 1)
p += CNOT(2, 3)
## QFT
p += SWAP(1, 2)
p += H(2)
p += CPHASE(-pi/2, 1, 2)
p += H(1)

### Controlled rotation Ry ###
p += X(1)
p += CRY(pi/8)(1, 0)
p += CRY(pi/16)(2, 0)

### Uncomputation ###
p += X(1)
p += H(1)
p += CPHASE(pi/2, 1, 2)
p += H(2)
p += SWAP(1, 2)
p += CNOT(2, 3)
p += PHASE(-pi, 1)
p += PHASE(-pi/2, 2)
p += H(2)
p += H(1)

Wavefunction Simulation (The two states that we need to pay attention to is the `0001` and `1001`):

In [4]:
from pyquil.api import WavefunctionSimulator
wfn = WavefunctionSimulator().wavefunction(p)
print(wfn)

(-0.0191221955+0j)|0000> + (0.0961337685+0j)|0001> + (0.9760625312+0j)|1000> + (0.1941509088+0j)|1001>


Run the circuit on a quantum virtual machine:

In [20]:
#### 1024 trials ####
p = Program() 
p += gate_definition

p += X(3) # Initilize input b

### Phase estimation ###
## Superposition
p += H(1)
p += H(2)
## Controlled-U
p += PHASE(pi/2, 2)
p += PHASE(pi, 1)
p += CNOT(2, 3)
## QFT
p += SWAP(1, 2)
p += H(2)
p += CPHASE(-pi/2, 1, 2)
p += H(1)

### Controlled rotation Ry ###
p += X(1)
p += CRY(pi/8)(1, 0)
p += CRY(pi/16)(2, 0)

### Uncomputation ###
p += X(1)
p += H(1)
p += CPHASE(pi/2, 1, 2)
p += H(2)
p += SWAP(1, 2)
p += CNOT(2, 3)
p += PHASE(-pi, 1)
p += PHASE(-pi/2, 2)
p += H(2)
p += H(1)

# run the program on a QVM
n_trials = 1024
qc = get_qc('9q-square-qvm')
result = qc.run_and_measure(p, trials=n_trials)
x1_square = 0
x2_square = 0
#print(result[0])
#print(result[1])
#print(result[2])
#print(result[3])
for i in range(n_trials):
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 0:
        x1_square += 1
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 1:
        x2_square += 1
p = x1_square/x2_square
print(x1_square)
print(x2_square)
print(p)  # Ideally, p=0.5²=0.25

10
41
0.24390243902439024


In [21]:
#### 2048 trials ####
p = Program() 
p += gate_definition

p += X(3) # Initilize input b

### Phase estimation ###
## Superposition
p += H(1)
p += H(2)
## Controlled-U
p += PHASE(pi/2, 2)
p += PHASE(pi, 1)
p += CNOT(2, 3)
## QFT
p += SWAP(1, 2)
p += H(2)
p += CPHASE(-pi/2, 1, 2)
p += H(1)

### Controlled rotation Ry ###
p += X(1)
p += CRY(pi/8)(1, 0)
p += CRY(pi/16)(2, 0)

### Uncomputation ###
p += X(1)
p += H(1)
p += CPHASE(pi/2, 1, 2)
p += H(2)
p += SWAP(1, 2)
p += CNOT(2, 3)
p += PHASE(-pi, 1)
p += PHASE(-pi/2, 2)
p += H(2)
p += H(1)

# run the program on a QVM
n_trials = 2048
qc = get_qc('9q-square-qvm')
result = qc.run_and_measure(p, trials=n_trials)
x1_square = 0
x2_square = 0
for i in range(n_trials):
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 0:
        x1_square += 1
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 1:
        x2_square += 1
p = x1_square/x2_square
print(x1_square)
print(x2_square)
print(p)  # Ideally, p=0.5²=0.25

21
75
0.28


In [22]:
#### 4096 trials ####
p = Program() 
p += gate_definition

p += X(3) # Initilize input b

### Phase estimation ###
## Superposition
p += H(1)
p += H(2)
## Controlled-U
p += PHASE(pi/2, 2)
p += PHASE(pi, 1)
p += CNOT(2, 3)
## QFT
p += SWAP(1, 2)
p += H(2)
p += CPHASE(-pi/2, 1, 2)
p += H(1)

### Controlled rotation Ry ###
p += X(1)
p += CRY(pi/8)(1, 0)
p += CRY(pi/16)(2, 0)

### Uncomputation ###
p += X(1)
p += H(1)
p += CPHASE(pi/2, 1, 2)
p += H(2)
p += SWAP(1, 2)
p += CNOT(2, 3)
p += PHASE(-pi, 1)
p += PHASE(-pi/2, 2)
p += H(2)
p += H(1) 

# run the program on a QVM
n_trials = 4096
qc = get_qc('9q-square-qvm')
result = qc.run_and_measure(p, trials=n_trials)
x1_square = 0
x2_square = 0
for i in range(n_trials):
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 0:
        x1_square += 1
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 1:
        x2_square += 1
p = x1_square/x2_square
print(x1_square)
print(x2_square)
print(p)  # Ideally, p=0.5²=0.25

29
155
0.1870967741935484


In [23]:
#### 8192 trials ####
p = Program() 
p += gate_definition

p += X(3) # Initilize input b

### Phase estimation ###
## Superposition
p += H(1)
p += H(2)
## Controlled-U
p += PHASE(pi/2, 2)
p += PHASE(pi, 1)
p += CNOT(2, 3)
## QFT
p += SWAP(1, 2)
p += H(2)
p += CPHASE(-pi/2, 1, 2)
p += H(1)

### Controlled rotation Ry ###
p += X(1)
p += CRY(pi/8)(1, 0)
p += CRY(pi/16)(2, 0)

### Uncomputation ###
p += X(1)
p += H(1)
p += CPHASE(pi/2, 1, 2)
p += H(2)
p += SWAP(1, 2)
p += CNOT(2, 3)
p += PHASE(-pi, 1)
p += PHASE(-pi/2, 2)
p += H(2)
p += H(1)

# run the program on a QVM
n_trials = 8192
qc = get_qc('9q-square-qvm')
result = qc.run_and_measure(p, trials=n_trials)
x1_square = 0
x2_square = 0
for i in range(n_trials):
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 0:
        x1_square += 1
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 1:
        x2_square += 1
p = x1_square/x2_square
print(x1_square)
print(x2_square)
print(p)  # Ideally, p=0.5²=0.25

73
305
0.23934426229508196


In [24]:
#### 10240 trials ####
p = Program() 
p += gate_definition

p += X(3) # Initilize input b

### Phase estimation ###
## Superposition
p += H(1)
p += H(2)
## Controlled-U
p += PHASE(pi/2, 2)
p += PHASE(pi, 1)
p += CNOT(2, 3)
## QFT
p += SWAP(1, 2)
p += H(2)
p += CPHASE(-pi/2, 1, 2)
p += H(1)

### Controlled rotation Ry ###
p += X(1)
p += CRY(pi/8)(1, 0)
p += CRY(pi/16)(2, 0)

### Uncomputation ###
p += X(1)
p += H(1)
p += CPHASE(pi/2, 1, 2)
p += H(2)
p += SWAP(1, 2)
p += CNOT(2, 3)
p += PHASE(-pi, 1)
p += PHASE(-pi/2, 2)
p += H(2)
p += H(1)

# run the program on a QVM
n_trials = 10240
qc = get_qc('9q-square-qvm')
result = qc.run_and_measure(p, trials=n_trials)
x1_square = 0
x2_square = 0
for i in range(n_trials):
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 0:
        x1_square += 1
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 1:
        x2_square += 1
p = x1_square/x2_square
print(x1_square)
print(x2_square)
print(p)  # Ideally, p=0.5²=0.25

79
383
0.206266318537859


In [25]:
#### 15360 trials ####
p = Program() 
p += gate_definition

p += X(3) # Initilize input b

### Phase estimation ###
## Superposition
p += H(1)
p += H(2)
## Controlled-U
p += PHASE(pi/2, 2)
p += PHASE(pi, 1)
p += CNOT(2, 3)
## QFT
p += SWAP(1, 2)
p += H(2)
p += CPHASE(-pi/2, 1, 2)
p += H(1)

### Controlled rotation Ry ###
p += X(1)
p += CRY(pi/8)(1, 0)
p += CRY(pi/16)(2, 0)

### Uncomputation ###
p += X(1)
p += H(1)
p += CPHASE(pi/2, 1, 2)
p += H(2)
p += SWAP(1, 2)
p += CNOT(2, 3)
p += PHASE(-pi, 1)
p += PHASE(-pi/2, 2)
p += H(2)
p += H(1)

# run the program on a QVM
n_trials = 15360
qc = get_qc('9q-square-qvm')
result = qc.run_and_measure(p, trials=n_trials)
x1_square = 0
x2_square = 0
for i in range(n_trials):
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 0:
        x1_square += 1
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 1:
        x2_square += 1
p = x1_square/x2_square
print(x1_square)
print(x2_square)
print(p)  # Ideally, p=0.5²=0.25

120
582
0.20618556701030927


In [26]:
#### 20480 trials ####
p = Program() 
p += gate_definition

p += X(3) # Initilize input b

### Phase estimation ###
## Superposition
p += H(1)
p += H(2)
## Controlled-U
p += PHASE(pi/2, 2)
p += PHASE(pi, 1)
p += CNOT(2, 3)
## QFT
p += SWAP(1, 2)
p += H(2)
p += CPHASE(-pi/2, 1, 2)
p += H(1)

### Controlled rotation Ry ###
p += X(1)
p += CRY(pi/8)(1, 0)
p += CRY(pi/16)(2, 0)

### Uncomputation ###
p += X(1)
p += H(1)
p += CPHASE(pi/2, 1, 2)
p += H(2)
p += SWAP(1, 2)
p += CNOT(2, 3)
p += PHASE(-pi, 1)
p += PHASE(-pi/2, 2)
p += H(2)
p += H(1)

# run the program on a QVM
n_trials = 20480
qc = get_qc('9q-square-qvm')
result = qc.run_and_measure(p, trials=n_trials)
x1_square = 0
x2_square = 0
for i in range(n_trials):
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 0:
        x1_square += 1
    if result[0][i] == 1 and result[1][i] == 0 and result[2][i] == 0 and result[3][i] == 1:
        x2_square += 1
p = x1_square/x2_square
print(x1_square)
print(x2_square)
print(p)  # Ideally, p=0.5²=0.25

210
774
0.2713178294573643
